# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [18]:
%%bash

sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Hit:2 https://packages.microsoft.com/repos/microsoft-ubuntu-focal-prod focal InRelease
Get:3 https://dl.yarnpkg.com/debian stable InRelease [17.1 kB]
Hit:4 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [30.9 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3711 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1214 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3671 kB]
Hit:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:12 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [385

Reading package lists...
Building dependency tree...
Reading state information...
snmp-mibs-downloader is already the newest version (1.2).
snmp is already the newest version (5.8+dfsg-2ubuntu2.9).
snmpd is already the newest version (5.8+dfsg-2ubuntu2.9).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [19]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [33]:
%%bash

sudo service snmpd restart
sudo service snmpd status

 * snmpd is running


## List all downloaded mibs

In [21]:
%%bash

ls -R /usr/share/snmp/mibs

/usr/share/snmp/mibs:
AIRCONDITIONER-MIB.txt
GNOME-SMI.txt
LM-SENSORS-MIB.txt
NET-SNMP-AGENT-MIB.txt
NET-SNMP-EXAMPLES-MIB.txt
NET-SNMP-EXTEND-MIB.txt
NET-SNMP-MIB.txt
NET-SNMP-MONITOR-MIB.txt
NET-SNMP-PASS-MIB.txt
NET-SNMP-PERIODIC-NOTIFY-MIB.txt
NET-SNMP-SYSTEM-MIB.txt
NET-SNMP-TC.txt
NET-SNMP-VACM-MIB.txt
UCD-DEMO-MIB.txt
UCD-DISKIO-MIB.txt
UCD-DLMOD-MIB.txt
UCD-IPFILTER-MIB.txt
UCD-IPFWACC-MIB.txt
UCD-SNMP-MIB-OLD.txt
UCD-SNMP-MIB.txt
iana
ietf


## Show RFC1213 object tree

In [22]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/AIRCONDITIONER-MIB.txt"

snmptranslate -m $MIB -Tp

Unlinked OID in AIRCONDITIONER-MIB: airConditioner ::= { experimental 99999 }
Undefined identifier: experimental near line 7 of /usr/share/snmp/mibs/AIRCONDITIONER-MIB.txt


+--iso(1)
   |
   +--org(3)
      |
      +--dod(6)
         |
         +--internet(1)
            |
            +--directory(1)
            |
            +--mgmt(2)
            |  |
            |  +--mib-2(1)
            |     |
            |     +--transmission(10)
            |
            +--experimental(3)
            |  |
            |  +--airConditioner(99999)
            |     |
            |     +--acMainObjects(1)
            |        |
            |        +-- -RW- EnumVal   acStatus(1)
            |        |        Values: off(0), on(1)
            |        +-- -R-- Integer32 acCurrentTemperature(2)
            |        |        Range: -30..50
            |        +-- -RW- Integer32 acDesiredTemperature(3)
            |        |        Range: -30..50
            |        +-- -RW- EnumVal   acMode(4)
            |        |        Values: auto(1), cool(2), heat(3), fan(4)
            |        +-- -RW- EnumVal   acFanSpeed(5)
            |        |        Values: low(1), mediu

## Show the description of a specific object

In [23]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

sysDescr OBJECT-TYPE
    SYNTAX  DisplayString (SIZE (0..255))
    ACCESS  read-only
    STATUS  mandatory
    DESCRIPTION
            "A textual description of the entity.  This value
            should include the full name and version
            identification of the system's hardware type,
            software operating-system, and networking
            software.  It is mandatory that this only contain
            printable ASCII characters."
    ::= { system 1 }


## Using snmpget

In [24]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

SNMPv2-MIB::sysDescr.0 = STRING: Linux codespaces-396979 6.5.0-1021-azure #22~22.04.1-Ubuntu SMP Tue Apr 30 16:08:18 UTC 2024 x86_64
SNMPv2-MIB::sysContact.0 = STRING: admin@localhost


## Using snmpset

In [25]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

SNMPv2-MIB::sysContact.0 = STRING: admin@localhost


SNMPv2-MIB::sysContact.0 = STRING: admin@localhost


## Using snmpgetnext

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Exercícios - SNMPv1

a) obter o endereço físico (MAC) da 2a. interface de rede da tabela ifTable

In [ ]:
%%bash

snmpget -v 1 -c public localhost ifPhysAddress.2

b) obter o número de mensagens ICMP enviadas e recebidas pelo elemento gerenciado

In [ ]:
%%bash

snmpget -v 1 -c public localhost icmpInMsgs.0 icmpOutMsgs.0

c) obter o número de requisições GET recebidos pelo agente no elemento gerenciado.

In [ ]:
%%bash

snmpget -v 1 -c public localhost snmpInGetRequests.0

d) modificar o nome do elemento gerenciado para "Gerencia"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysName.0 s "Gerencia"

e) modificar a identificação da localização do elemento gerenciado para "LabRedes"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysLocation.0 s "LabRedes"

f) obter a descrição da interface localizada na 1a. linha da tabela ifTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ifDescr

g) obter o endereço IP, índice da interface na tabela ifTable, máscara de rede e endereço de broadcast da primeira entrada da tabela ipAddrEntry

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ipAdEntIfIndex ipAdEntNetMask ipAdEntBcastAddr

h) obter o endereço local e porta local da primeira linha da tabela udpTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost udpLocalPort udpLocalAddress

## Using snmpbulkget

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn2 -Cr3 system interfaces ifIndex ifDescr ifSpeed

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn4 -Cr5 tcpInSegs tcpOutSegs tcpRetransSegs tcpInErrs tcpConnState tcpConnLocalAddress tcpConnLocalPort tcpConnRemAddress tcpConnRemPort

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn0 -Cr4 udpLocalAddress udpLocalPort

## Comparing snmpwalk vs snmpbulkwalk



In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## snmpd.conf

In [32]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.99999.1.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent
sudo service snmpd restart
sudo service snmpd status

 * snmpd is running


## Agent code

In [31]:
%%writefile /tmp/agent.py

#!/usr/bin/env python3

import sys
import datetime
import socket

# Variáveis para simular o estado do ar condicionado
ac_status = 1  # 0: off, 1: on
ac_current_temperature = 22  # Temperatura atual em Celsius
ac_desired_temperature = 24  # Temperatura desejada em Celsius
ac_mode = 2  # 1: auto, 2: cool, 3: heat, 4: fan
ac_fan_speed = 3  # 1: low, 2: medium, 3: high
ac_error_message = "errooooo"  # Mensagem de erro, se houver

def get_ac_status():
    return ac_status

def get_ac_current_temperature():
    return ac_current_temperature

def get_ac_desired_temperature():
    return ac_desired_temperature

def get_ac_mode():
    return ac_mode

def get_ac_fan_speed():
    return ac_fan_speed

def get_ac_error_message():
    return ac_error_message

def set_ac_desired_temperature(value):
    global ac_desired_temperature
    ac_desired_temperature = value
    return ac_desired_temperature

def main():

    with open("/tmp/agent.log", 'w') as file:
        file.write(' '.join(sys.argv))

    if len(sys.argv) != 3:
        print("Usage: agent.py <MIB-oid> <request-type>")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.99999.1.1.1":
            print(".1.3.6.1.3.99999.1.1.1")
            print("integer")
            print(get_ac_status())
        elif oid == ".1.3.6.1.3.99999.1.1.2":
            print(".1.3.6.1.3.99999.1.1.1.2")
            print("integer")
            print(get_ac_current_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.3":
            print(".1.3.6.1.3.99999.1.1.3")
            print("integer")
            print(get_ac_desired_temperature())
        elif oid == ".1.3.6.1.3.99999.1.1.4":
            print(".1.3.6.1.4.1.99999.1.1.4")
            print("integer")
            print(get_ac_mode())
        elif oid == ".1.3.6.1.3.99999.1.1.5":
            print(".1.3.6.1.4.1.99999.1.1.5")
            print("integer")
            print(get_ac_fan_speed())
        elif oid == ".1.3.6.1.3.99999.1.1.6":
            print(".1.3.6.1.4.1.99999.1.1.6")
            print("string")
            print(get_ac_error_message())
        else:
            print("NONE")
    elif request_type == "-s":  # SET request
        if oid == ".1.3.6.1.3.99999.1.1.3":
            try:
                value = int(sys.argv[4])
                result = set_ac_desired_temperature(value)
                print(".1.3.6.1.4.1.99999.1.1.3")
                print("integer")
                print(result)
            except ValueError:
                print("Invalid value")
        else:
            print("NONE")
    else:
        print("NONE")

if __name__ == "__main__":
    main()


Overwriting /tmp/agent.py


## Testing

In [36]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.999.1.1.0
snmpget -v2c -c public localhost .1.3.6.1.3.999.1.1.1

SNMPv2-SMI::experimental.999.1.1.0 = No Such Object available on this agent at this OID
SNMPv2-SMI::experimental.999.1.1.1 = No Such Object available on this agent at this OID
